# Coursera Capstone Project.

In [273]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [274]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Get Wikipedia HTML Page

In [275]:
!wget -O Canada.html https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada%3A_M

--2019-06-03 09:03:08--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada%3A_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78947 (77K) [text/html]
Saving to: ‘Canada.html’

Canada.html         100%[===================>]  77.10K  --.-KB/s    in 0.05s   

2019-06-03 09:03:09 (1.60 MB/s) - ‘Canada.html’ saved [78947/78947]



In [276]:
with open("Canada.html") as fp:
   WikiCanada_bs = BeautifulSoup(fp, 'lxml')          # parser lxml or html.parser

## Get the table

In [277]:
tbl = WikiCanada_bs.find('tbody')

## Get all rows of the table and store them as a List

In [278]:
rows = tbl.find_all('tr')

In [279]:
can_Lst = []

for i, row in enumerate(rows):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    can_Lst.append(cols)


## Convert to a Pandas Dataframe and Set Column Names

In [280]:
can_df = pd.DataFrame( can_Lst[1:] )

In [281]:
can_df.head(3)

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


In [282]:
can_df.columns = ['PostalCode','Borough','Neighborhood']
can_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Print number of rows of the original data

In [283]:
can_df.shape

(288, 3)

## Replace 'Not Assign' with NaN

In [284]:
can_df.replace('Not assigned', np.nan, inplace = True)

## Set Neighborhood with Borough name when Neighborhood is NaN 

In [285]:
can_df['Neighborhood'].replace( np.nan,  can_df['Borough'], inplace = True )

## Confirm if successful, check M7A

In [286]:
can_df[ can_df['PostalCode'] == 'M7A'  ]

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Queen's Park


## Remove Row with NaN

In [287]:
can_df.dropna(axis = 0, inplace = True)
can_df.reset_index(drop=True, inplace=True)
can_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Check again the number of Row after deletion

In [288]:
can_df.shape

(211, 3)

## Display Neighborhoods of M5A

In [289]:
can_df[ can_df['PostalCode'] == 'M5A'  ]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park


# The code Below will merge all Neighborhood from the same PostalCode

## Use Dictionary to Merge Neigborhood
###   Postal Code + '.' + Borough will be the key of the dictionary

In [290]:
can_dict = {}

for vPostal, vBor, vNeigh in zip( Can_df.PostalCode, Can_df.Borough, Can_df.Neighborhood):
    vKey = vPostal + "." + vBor
    try:
        can_dict[ vKey ].append(vNeigh) 
    except:
        can_dict[ vKey ] = [vNeigh]
    

## Split the dictionary to 3 seperate lists

In [291]:
can_pc = []
can_bo = []
can_ng = []
for vDict, vVal in zip( can_dict.keys(), can_dict.values() ):
    v_pc_bo = vDict.split('.') 
    can_pc.append( v_pc_bo[0] )
    can_bo.append( v_pc_bo[1] )
    can_ng.append( ','.join(vVal) )


## Convert the List to a Pandas Dataframe

In [292]:
can_df2 = pd.DataFrame( {'PostalCode':can_pc , 'Borough':can_bo, 'Neighborhood':can_ng} )
can_df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M9A,Etobicoke,Islington Avenue


## Check the number of rows of the final result

In [293]:
can_df2.shape

(102, 3)